## 第二部

- 今まで
    - 計算をモデル化してきた
        - ch2: SIMPLE
        - ch3: DFA/NFA
        - ch4: DPDA/NPDA
        - ch5: DTM
    - 計算ってなんだろう？を考えてきた
         - 質問に答えるもの
         - 入力に対し出力返すもの
    - 人間がわかりやすい形での理解だった
- こっから
    - 人間の理解しやすい形**ではない**形式で考えていく

## chapter 6
- ゼロから計算の概念を組み立てていく
- 型なしラムダ計算
    - ch6.1: rubyのprocを用いてlambda計算（っぽいこと）をする
    - ch6.2: lambda計算を実装する

## 6.1: ラムダ計算をまねる
- 最小限の機能でプログラムを書こう
- 使用禁止
    - gem
    - std lib
    - module
    - method
    - class
    - object
    - 制御構造
    - 代入
    - 配列
    - 文字列
    - 数値
    - ブール値
- 使用可能
    - 変数参照
        - aliasとしての変数利用
        - ?代入禁止とconflictするのでは
    - proc生成
    - proc呼び出し
    
- 要するに：論理構造，データ構造などすべて禁止．自前でつくっていく
- こんな感じですべてを表す

### 6.1.1: procのポイント

procってなにができるの

In [1]:
#proc
-> x { -> { x.call(y)}}

#<Proc:0x007fc79d82eff8@<main>:1 (lambda)>

#### 6.1.1.1 配管

callできるよ

In [2]:
# call
-> x {x + 2}.call(1)

3

#### 6.1.1.2 引数

複数引数取れるよ

In [3]:
-> x,y {x + y}.call(3,4)

7

でも別に，複数引数撮れることはmustじゃないよ

仮にprocが単引数しか許容しなかったとしても，同じことは出来るよ

In [4]:
-> x {
  -> y {
    x + y
  }
}.call(3).call(4)

7

この場合，引数を少なく返すと「残りの引数を受け取るproc」が返るよ

In [5]:
# 2引数のところ1引数で渡すと..
addThree = -> x {
  -> y {
    x + y
  }
}.call(3)

#<Proc:0x007fc79d01f290@<main>:2 (lambda)>

In [6]:
# 引数に3をたす関数になった
addThree.call(4)

7

In [7]:
addThree.call(10)

13

いわゆるカリー化である

#### 6.1.1.3 等価性

内部定義が違っても，同じ呼び出しに同じ結果を返すなら等価である
外延等価性というらしい

以下は等価である

In [8]:
p = -> n {n * 2}

#<Proc:0x007fc79c82c600@<main>:0 (lambda)>

In [9]:
q = -> x { p.call(x)}

#<Proc:0x007fc79d1384d8@<main>:0 (lambda)>

In [10]:
puts p.call(10)
puts q.call(10)

20
20


等価であれば，交換可能であるのは自然の理

```
p = q
p = -> x { p.call(x)}
```
すなわち，

```
-> x {p.call(x)}
```
なるパターンを見つけたら無条件で `p` に書き換えてok

#### 6.1.1.4 構文

引数はこんな形でも与えられるよ

In [11]:
-> x {x + 5}[10]

15

### 6.1.2 問題

FizzBuzzを先の制約のもとかいてみよう

まずは普通にrubyつかってFizzBuzzかいて，それをstep by stepで変換していくよ

In [12]:
# いわゆるFizzBuzz
(1..100).map do |n|
  if (n % 15).zero?
    'FizzBuzz'
  elsif (n % 3).zero?
    'Fizz'
  elsif (n % 5).zero?
    'Buzz'
  else
    n
  end
end

[1, 2, "Fizz", 4, "Buzz", "Fizz", 7, 8, "Fizz", "Buzz", 11, "Fizz", 13, 14, "FizzBuzz", 16, 17, "Fizz", 19, "Buzz", "Fizz", 22, 23, "Fizz", "Buzz", 26, "Fizz", 28, 29, "FizzBuzz", 31, 32, "Fizz", 34, "Buzz", "Fizz", 37, 38, "Fizz", "Buzz", 41, "Fizz", 43, 44, "FizzBuzz", 46, 47, "Fizz", 49, "Buzz", "Fizz", 52, 53, "Fizz", "Buzz", 56, "Fizz", 58, 59, "FizzBuzz", 61, 62, "Fizz", 64, "Buzz", "Fizz", 67, 68, "Fizz", "Buzz", 71, "Fizz", 73, 74, "FizzBuzz", 76, 77, "Fizz", 79, "Buzz", "Fizz", 82, 83, "Fizz", "Buzz", 86, "Fizz", 88, 89, "FizzBuzz", 91, 92, "Fizz", 94, "Buzz", "Fizz", 97, 98, "Fizz", "Buzz"]

このコードは使っちゃいけないものばかり使っているね

- 数
- ブール値
- 制御構造(if/elsif/else)
- 演算
- リスト( (1..100))
- 文字列
- etc.

ひとつずつ，すべてprocで表現していくよ

### 6.1.3 数

- 「数」という概念を「1,2,3..」を使わずに表現するよ
- 「数」ってなんだ？
    - ０の次に１
    - 1の次に２
    - ２の次に3..
- 1とか2のラベルに意味は無い
    - 表現の形でしか無いからね
    - 本質は「連続・反復を表す概念である」ということ
    - 我々は 0->1->2 という連続性を知っているので，これらのラベルは「数」を表現できる
        - その意味では「あ，い，う，え，お」とかも一緒
        - 実際16進数だとA,B..Fつかってるよね
- 「連続・反復」の概念を表現できれば，数を表現できる
    - 0: なにもしないこと
    - 1: アクションを行うこと
    - 2: アクションを行った後，アクションを行うこと
    - 3: アクションを行った後，アクションを行った後，アクションを行うこと
    - 以下略
    - 再帰っぽいな！

In [13]:
# ruby風に数を表現
def zero (proc, x)
  x
end

def one (proc, x)
  proc[x]
end

def two (proc, x)
  proc[proc[x]]
end

:two

よーするに：引数xに対しproc p を「何度」適用するか？で数を表す

In [14]:
# procで同じことをやると
ZERO = -> p { -> x { x}}
ONE = -> p { -> x {p[x]}}
TWO = -> p { -> x {p[p[x]]}}

#<Proc:0x007fc79d905a08@<main>:3 (lambda)>

スッキリかけて気持ちいいね！でも頭がこんがらがってきそうだね！

- このままだと本当にZEROが0, ONEが1なのか？がわかりづらい
- 人間が理解しやすいように，これらの値をruby世界の「数」に変換する関数をつくろう

In [15]:
# ZERO/ONE/..を0/1/..に変換する
def to_integer(proc)
  proc[-> x { x+1}][0]
end

:to_integer

In [16]:
puts to_integer(ZERO)
puts to_integer(ONE)
puts to_integer(TWO)

0
1
2


これで，FizzBuzzにひつような数 3, 5, 15, 100が定義できる！

In [17]:
THREE = -> p { -> x {p[p[p[x]]]}}
FIVE = -> p{ -> x {p[p[p[p[p[x]]]]]}}
FIFTEEN = -> p{ -> x {p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[x]]]]]]]]]]]]]]]}}
HUNDRED = -> p{ -> x {p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[p[x]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]}}

#<Proc:0x007fc79c053f00@<main>:3 (lambda)>

In [18]:
puts to_integer(THREE)
puts to_integer(FIVE)
puts to_integer(FIFTEEN)
puts to_integer(HUNDRED)

3
5
15
100


以上を用いてFizzBuzzはこうなります

In [19]:
#  数をprocで表現
(ONE..HUNDRED).map do |n|
  if (n % FIFTEEN).zero?
    'FizzBuzz'
  elsif (n % THREE).zero?
    'Fizz'
  elsif (n % FIVE).zero?
    'Buzz'
  else
    n
  end
end

ArgumentError: bad value for range

ただし，まだ独自の「数」に対して % とか .. とかを定義していないので，上のプログラムは動きません

### 6.1.4 ブール値

- ブール値ってなんだ？
    - True/False
    - ２つの選択肢から一つを選ぶもの
        - if bool then true_statement else false_statement
    - 最初から true_statement, false_statement のどちらかを返すもの，と定義できる

In [20]:
# rubyふうに
def true(x,y)
  x
end

def false(x,y)
  y
end

:false

In [21]:
# procふうに
# TRUE/FALSEは予約語っぽいorz
MY_TRUE = -> x { -> y { x }}
MY_FALSE = -> x { -> y { y }}

#<Proc:0x007fc79d1d8dc0@<main>:3 (lambda)>

変換も簡単

In [22]:
def to_boolean(proc)
  proc[true][false]
end

:to_boolean

In [23]:
puts to_boolean(MY_TRUE)
puts to_boolean(MY_FALSE)

true
false


合せて制御構造 if/elsif/else も定義しよう

In [24]:
# ruby ふうに
def if_(proc, x, y)
  proc[x][y]
end

:if_

In [25]:
# procふうに
IF = -> b { -> x { -> y { b[x][y]}}}

#<Proc:0x007fc79d0f1d08@<main>:1 (lambda)>

ここで procの変換ルールを思い出そう

`-> x { p[x]} ` == `p` だった

これをIFに適用

```
-> b {-> x {-> y { b[x][y]}}}
-> b {-> x {        b[x]      }}
-> b {              b          }}
```

そもそも true/falseを「２つの選択肢から一つを選ぶもの」として定義したので，ごく当たり前の話ではあるね

In [26]:
# 数，bool値をprocで
(ONE..HUNDRED).map do |n|
  IF[(n % FIFTEEN).zero?][
    'FizzBuzz'
  ][IF[(n % THREE).zero?][
    'Fizz'
  ][IF[(n % FIVE).zero?][
    'Buzz'
  ][
    n
  ]]]
end

ArgumentError: bad value for range

### 6.1.5 述語

- FizzBuzzをつくるためには，余りがゼロかの判定が必要（.zero?）
- 我々は「数」を「アクションを行う回数」と定義した
- 「常にfalse返すproc」をtrueに何度も適用する，とかんがえる

In [27]:
# rubyふう
def zreo?(proc)
  proc[->x{FALSE}][TRUE]
end

:zreo?

In [28]:
# procふう
IS_ZERO = -> p { p[ -> x {MY_FALSE}][MY_TRUE]}

#<Proc:0x007fc79c942a80@<main>:1 (lambda)>

In [29]:
puts to_boolean( IS_ZERO[ZERO])
puts to_boolean( IS_ZERO[ONE])

true
false


In [30]:
# 数,bool値,述語をprocで
(ONE..HUNDRED).map do |n|
  IF[IS_ZERO[(n % FIFTEEN)]][
    'FizzBuzz'
  ][IF[IS_ZERO[(n % THREE)]][
    'Fizz'
  ][IF[IS_ZERO[(n % FIVE)]][
    'Buzz'
  ][
    n
  ]]]
end

ArgumentError: bad value for range

### 6.1.6 ペア
- プリミティブとして数値，ブール値を定義した
- データ構造が欲しくなる
- ペアを定義する

- 要件
    - 2つの要素をもつ
    - left で1つ目の要素を取得
    - right で2つ目の要素を取得

In [31]:
# procふうに
PAIR = -> x { -> y { -> f { f[x][y] }}}
LEFT = -> p { p[ -> x { -> y {x}}]} 
RIGHT = -> p { p[-> x { -> y {y}}]} 

#<Proc:0x007fc79c8e8f80@<main>:3 (lambda)>

パッと見難しいけど，落ち着いて考えればやってることは簡単

- pair : procを受け取り，「予め受け取っていた二数x,y」を適用するproc
- left: procを受け取り，「二数x,yを受け取りxを返すproc」を適用するproc
- right: procを受け取り，「二数x,yを受け取りyを返すproc」を適用するproc

### 6.1.7 数値演算

- 各種データ型がそろってきた
- 演算を定義する

- まずはインクリメント
- 数 = p,x を受け取ってxにpを指定回数適用するもの
- インクリメント: nに更にpを適用する

In [32]:
INCREMENT = -> n { -> p { -> x {p[n[p][x]]}}}

#<Proc:0x007fc79c8ce5b8@<main>:0 (lambda)>

- デクリメント
- n回適用された時，一回戻ったものを返す
    - むずかしい
- pairを利用，slideという概念の導入


In [33]:
def slide(pair)
  [pair.last, pair.last+1]
end

# 一つずつずれたものになる
puts slide([3,4])
puts slide([7,8])

# ぶっちゃけ重要なの後ろの数だけ
puts slide([0, 4])
puts slide([10000, 4])

[4, 5]
[8, 9]
[4, 5]
[4, 5]


- 数n: xにpをn回適用するもの
- [0,0]にslideをn回適用する -> [n-1, n]
- left取ればdecrementできてる！

In [34]:
SLIDE = -> p {PAIR[RIGHT[p]][INCREMENT[RIGHT[p]]]}
DECREMENT = -> n { LEFT[n[SLIDE][PAIR[ZERO][ZERO]]]}

#<Proc:0x007fc79d190cc8@<main>:1 (lambda)>

In [35]:
puts to_integer(DECREMENT[ONE])
puts to_integer(DECREMENT[FIVE])
puts to_integer(DECREMENT[HUNDRED])

0
4
99


- 以上を用いれば四則演算が定義できるよ
- カリー化してることに注目するとわかりやすいかも

In [36]:
ADD = -> m { -> n { n[INCREMENT][m] }}
SUBTRACT = -> m { -> n {n[DECREMENT][m]}}
MULTIPLY = -> m { -> n {n[ADD[m]][ZERO]}}
POWER = -> m { -> n { n[MULTIPLY[m]][ONE]}}

#<Proc:0x007fc79d161ef0@<main>:3 (lambda)>

In [37]:
# こう考えるとわかりやすいかも

# 「5を足す」というproc
ADD_FIVE = FIVE[INCREMENT]

# nを受け「nを足す」というprocを返すproc
ADD_N = -> n {n[INCREMENT]}

#  mにnを受け「nを足す」というprocを返すprocを適用

ADD_N_TO_M = -> m { -> n {ADD_N[n][m]}}

#<Proc:0x007fc79d84ed08@<main>:10 (lambda)>

In [38]:
# +α

# add，これでよくね？
MYADD = -> m {-> n { -> p { -> x {m[p][n[p][x]]}}}}
puts to_integer(MYADD[TWO][FIVE])

# mul,　これでよくね？
MYMUL = -> m {-> n { -> p {m[n[p]]}}}
puts to_integer(MYMUL[TWO][FIVE])

7
10


- チラ裏memo

- Haskell風に書くとしんぷるでしあわせ
- decrementもねー，f^-1を適用する，って考え方のほうががいいよねー
    - `decrement n p = inverse p $ n p` みたいなー
    - fが全単射だったら，の話になっちゃうけどさー
    - いや，全射である必要はないか？

- あとは剰余（%）がほしい

In [39]:
def my_mod(m,n)
  if n <= m
    mod(m-n, n)
  else
    m
  end
end

:my_mod

- 割る数nを割られる数mから引きまくって，mが以下になったらreturnする
- なんてことはない
- 比較演算子(<=)を定義できれば行けそうだ!

```
m <= n
m - n <= 0
```

- 数m-nと0の比較ができれば，<=を定義できたことになる
- 我々の「数」においては，ゼロ以下の数は存在しない
    - decrement の定義より
- すなわち「aがゼロ以下」のとき，「a==0」となる
- すでにIS_ZEROは定義済みなので，<=が定義できそうだ

In [40]:
# m <= n
IS_LESS_OR_EQUAL = -> m {-> n { IS_ZERO[SUBTRACT[m][n]]}}

#<Proc:0x007fc79c826840@<main>:1 (lambda)>

In [41]:
puts to_boolean(IS_LESS_OR_EQUAL[FIVE][ONE])
puts to_boolean(IS_LESS_OR_EQUAL[ONE][FIVE])

false
true


- IS_LESS_OR_EQUALを用いてMODを定義しよう

In [42]:
MOD = -> m { -> n {
    IF[IS_LESS_OR_EQUAL[n][m]][
      MOD[SUBTRACT[m][n]][n]
      ][
      m
    ]
}}

#<Proc:0x007fc79d102130@<main>:0 (lambda)>

- ただしこのままでは動かない
- rubyのifは遅延評価だが．IFは正格評価のため
    - cond評価の前にthen/else節を評価する
    - 無限にthen節が呼ばれる
    - `MOD[SUBTRACT[ZERO][n]][n]` の繰り返しである
- then 節を遅延させたい
    - procでくるめばok
    - 等価性を思い出す
    - `p` = `->x {p[x]}`
    

In [43]:
MYMOD = -> m { -> n {
    IF[IS_LESS_OR_EQUAL[n][m]][
      -> x {
        MYMOD[SUBTRACT[m][n]][n][x]
        }
      ][
        m
        ]
}}

#<Proc:0x007fc79d084988@<main>:0 (lambda)>

In [44]:
puts to_boolean(IS_LESS_OR_EQUAL[FIVE][TWO])
puts to_integer(MYMOD[THREE][TWO])

false
1


- このコードはMYMODを変数として再帰のために利用している
- 代入の機能をつかっているではないか！
- 再帰関数定義のためのprocがひつよう
- 通称Yコンビネータ
     - よーするに：fを無限に繰り返します
- 手を動かして考えたほうがわかると思う

In [45]:
Y = -> f { -> x { f[x[x]] }[-> x { f[x[x]] }] }

#<Proc:0x007fc79d035e28@<main>:0 (lambda)>

- これも無限評価となるので遅延verにする必要がある
- pointは `x[x]`による再帰呼び出し

In [46]:
Z = -> f { -> x { f[-> y { x[x][y] }] }[-> x { f[-> y { x[x][y] }] }] }

#<Proc:0x007fc79d1ebe20@<main>:0 (lambda)>

In [47]:
ZMOD = 
  Z[-> f {-> m { -> n {
    IF[IS_LESS_OR_EQUAL[n][m]][
      -> x {
        f[SUBTRACT[m][n]][n][x]
        }
      ][
        m
      ]
}}}]

#<Proc:0x007fc79d1d9590@<main>:1 (lambda)>

In [48]:
puts to_integer(ZMOD[THREE][TWO])

1


In [49]:
# 数,bool値,述語,%をprocで
(ONE..HUNDRED).map do |n|
  IF[IS_ZERO[ZMOD[n][FIFTEEN]]][
    'FizzBuzz'
  ][IF[IS_ZERO[ZMOD[n][THREE]]][
    'Fizz'
  ][IF[IS_ZERO[ZMOD[n][FIVE]][
    'Buzz'
  ][
    n.to_s
  ]]]
end

SyntaxError: <main>:11: syntax error, unexpected keyword_end, expecting ']'

- チラ裏memo
- Zコンビネータに渡すprocにfが増えるのなんじゃい?
- おそらく自分自身を指すものだろう
- コンビネータに渡された関数は f[f[f...]] みたいになるはず
- fは自分自身なので，内側カッコからは `m -> SUBTRACT[m][n], n->n` で自身が呼ばれるのではないか？
- 一周目fにおいて，第一引数fはf[f[..]]となる(ややこしや
- 遅延評価なので渡されさえすれば一周目ロジックはうごかせる
- IS_LESS_OR_EQUALがtrueの時のみ，再びfの呼び出しが評価される
    - このときm,が更新されるので，いずれIS_LESS_OR_EQUALがfalse隣，無碍に呼び出しが終わる
- 第一引数は自身の無限呼び出しをするprocが入っており，何度呼ばれようが自身が返る．みたいなイメージだな

### 6.1.8 リスト

- 詳細略
- 雰囲気つかめれば十分
- らしい

- ようするに
    - PAIRつかって連結リストっぽいもの作ってる
    - ただし，IS_EMPTY対応のため．常にLEFTにTRUE/FALSEを入れたPAIRでくるむ

In [50]:
EMPTY = PAIR[MY_TRUE][MY_TRUE]
UNSHIFT = -> l {-> x {PAIR[MY_FALSE][PAIR[x][l]]}}
IS_EMPTY = LEFT
FIRST = -> l {LEFT[RIGHT[l]]}
REST = -> l {RIGHT[RIGHT[l]]}

#<Proc:0x007fc79d17bb98@<main>:4 (lambda)>

In [51]:
# 本と順番変えた．こっちのほうがわかりやすいと思う
mylist =UNSHIFT[UNSHIFT[UNSHIFT[EMPTY][ONE]][TWO]][THREE]

puts to_integer(FIRST[mylist])
puts to_integer(FIRST[REST[mylist]])
puts to_integer(FIRST[REST[REST[mylist]]])

puts to_boolean(IS_EMPTY[mylist])
puts to_boolean(IS_EMPTY[EMPTY])

3
2
1
false
true


In [52]:
# array表示のためのmethod
def to_array(proc)
    array = []
  
    until to_boolean(IS_EMPTY[proc])
        array.push(FIRST[proc])
        proc = REST[proc]
    end
    
    array
end

:to_array

In [53]:
puts to_array(mylist).map { |p| to_integer(p)}

[3, 2, 1]


- rangeを実装しよう

In [54]:
# rubyふう
def my_range(m, n)
  if m <= n
    my_range(m+1,n).unshift(m)
  else
    []
  end
end

puts my_range(0, 10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [55]:
# procで

RANGE = 
Z[->f {
  ->m { -> n {
    IF[IS_LESS_OR_EQUAL[m][n]][
      ->x{UNSHIFT[f[INCREMENT[m]][n]][m][x]}
      ][
        EMPTY
        ]
    }}
  }]

#<Proc:0x007fc79d90d820@<main>:4 (lambda)>

In [56]:
my_array = RANGE[ONE][FIVE]
puts to_array(my_array).map { |p| to_integer(p)}

[1, 2, 3, 4, 5]


In [57]:
# FizzBuzz
# 数,bool値,述語,%,rangeをprocで
RANGE[ONE]][HUNDRED].map do |n|
  IF[IS_ZERO[ZMOD[n][FIFTEEN]]][
    'FizzBuzz'
  ][IF[IS_ZERO[ZMOD[n][THREE]]][
    'Fizz'
  ][IF[IS_ZERO[ZMOD[n][FIVE]][
    'Buzz'
  ][
    n.to_s
  ]]]
end

SyntaxError: <main>:2: syntax error, unexpected ']', expecting end-of-input
RANGE[ONE]][HUNDRED].map do |n|
           ^

- mapもprocで定義しよう
- helperとしてFOLDを用いる
    - Haskellでいうfoldlっぽさ

In [58]:
# FOLD :: [a] -> b ->  (a->b->b) -> b
FOLD =
Z[-> f {
  -> l { -> x { -> g {
    IF[IS_EMPTY[l]][
      x
      ][
        -> y { 
          g[f[REST[l]][x][g]][FIRST[l]][y]
        }
     ]
    }}}
  }]

# f :: [a] -> b -> (a->b->b) -> b
# g  :: x -> x -> x

#<Proc:0x007fc79d16bba8@<main>:2 (lambda)>

In [59]:
puts to_integer(FOLD[RANGE[ONE][FIVE]][ZERO][ADD])
puts to_integer(FOLD[RANGE[ONE][FIVE]][ONE][MULTIPLY])

15
120


In [60]:
# MAP :: [x] -> (x -> x) -> [x]
MAP = 
-> k { -> f {
  FOLD[k][EMPTY][
   ->l { -> x {UNSHIFT[l][f[x]]}}  
  ]
}}

#<Proc:0x007fc79d916510@<main>:2 (lambda)>

In [61]:
my_list = MAP[RANGE[ONE][FIVE]][INCREMENT]
puts to_array(my_list).map { |p| to_integer(p) }

[2, 3, 4, 5, 6]


In [62]:
# FizzBuzz
# 数,bool値,述語,%,range,mapをprocで
MAP[RANGE[ONE][HUNDRED]][ -> {
  IF[IS_ZERO[ZMOD[n][FIFTEEN]]][
    'FizzBuzz'
  ][IF[IS_ZERO[ZMOD[n][THREE]]][
    'Fizz'
  ][IF[IS_ZERO[ZMOD[n][FIVE]]][
    'Buzz'
  ][
    n.to_s
  ]]]
}]

#<Proc:0x007fc79d055700@<main>:6 (lambda)>

### 6.1.9 文字列
- FizzBuzzで使う文字だけをencodeする形とする
- 数字も文字列としてreturnする

In [63]:
TEN = MULTIPLY[TWO][FIVE]
B = TEN
F = INCREMENT[B]
I = INCREMENT[F]
U = INCREMENT[I]
ZED = INCREMENT[U] 

FIZZ = UNSHIFT[UNSHIFT[UNSHIFT[UNSHIFT[EMPTY][ZED]][ZED]][I]][F]
BUZZ = UNSHIFT[UNSHIFT[UNSHIFT[UNSHIFT[EMPTY][ZED]][ZED]][U]][B]
FIZZBUZZ = UNSHIFT[UNSHIFT[UNSHIFT[UNSHIFT[BUZZ][ZED]][ZED]][I]][F]

#<Proc:0x007fc79d0262e8@<main>:1 (lambda)>

In [64]:
def to_char(c)
  '0123456789BFiuz'.slice(to_integer(c))
end

def to_string(s)
  to_array(s).map { |c| to_char(c) }.join
end


:to_string

In [65]:
puts to_char(ZED)
puts to_string(FIZZBUZZ)

z
FizzBuzz


In [66]:
# FizzBuzz
# 数,bool値,述語,%,range,map,文字列をprocで
MAP[RANGE[ONE][HUNDRED]][ -> {
  IF[IS_ZERO[ZMOD[n][FIFTEEN]]][
    FIZZBUZZ
  ][IF[IS_ZERO[ZMOD[n][THREE]]][
    FIZZ
  ][IF[IS_ZERO[ZMOD[n][FIVE]]][
    BUZZ
  ][
    n.to_s
  ]]]
}]

#<Proc:0x007fc79c8be3e8@<main>:6 (lambda)>

- 数値を文字列にする


In [67]:
# ruby でかく
def to_digits(n)
  previous_digits =
    if n < 10
      []
    else
      to_digits(n / 10)
    end
  previous_digits.push(n % 10) \
end

:to_digits

In [68]:
puts to_digits(123)

[1, 2, 3]


- to_digits をproc化
- < の代わりに<=
- /とpushを実装する
- DIVちょっとわかりづらいかも
    - どんな形で展開されていくか？を意識できればok
    - 要は再帰的に呼ばれるたびにINCREMENTでくるんでいる
    - 最終形
        - INCREMENT[INCREMENT[...INCREMENT[ZERO]..]]
- PUSHは簡単
    - [x]に対しFOLDでUNSHIFTしまくる

In [69]:
# DIV :: x -> x -> x
DIV =
Z[
    -> f { -> m { -> n {
      IF[IS_LESS_OR_EQUAL[n][m]][
          ->x {
            INCREMENT[f[SUBTRACT[m][n]][n]][x]}
      ][
          ZERO
      ]
   }}}  
]

# PUSH :: [x] -> x -> [x]
PUSH = -> l {
  -> x { 
    FOLD[l][UNSHIFT[EMPTY][x]][UNSHIFT]
    }
  }

# TO_DIGITS :: x -> [x]
TO_DIGITS = 
Z[ -> f {-> n { PUSH[
  IF[IS_LESS_OR_EQUAL[n][DECREMENT[TEN]]][
    EMPTY
  ][
    -> x {
        f[DIV[n][TEN]][x]
    }    
  ]
][ZMOD[n][TEN]]}}]

#<Proc:0x007fc79c9326d0@<main>:22 (lambda)>

In [70]:
puts to_array(TO_DIGITS[FIFTEEN]).map { |p| to_integer(p) }

[1, 5]


In [71]:
# FizzBuzz
# 数,bool値,述語,%,range,map,文字列をprocで
ANS = MAP[RANGE[ONE][HUNDRED]][ -> n {
  IF[IS_ZERO[ZMOD[n][FIFTEEN]]][
    FIZZBUZZ
  ][IF[IS_ZERO[ZMOD[n][THREE]]][
    FIZZ
  ][IF[IS_ZERO[ZMOD[n][FIVE]]][
    BUZZ
  ][
    TO_DIGITS[n]
  ]]]
}]

#<Proc:0x007fc79d845690@<main>:6 (lambda)>

In [72]:
to_array(ANS).each do |p|
    puts to_string(p) 
end; nil


1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz


꒰(๑´•.̫ • `๑)꒱< FIZZBUZZ できてる☆